구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


3-1

***User-based Filtering (사용자 기반 필터링)***
: 비슷한 선호도를 가진 응답자들이 특정 스타일을 선호/ 비선호 하는지를 바탕으로 새로운 응답자가 이들을 선호/ 비선호 스타일을 추천해주는 방식

**적용 방법**: 각 응답자의 스타일 선호/ 비선호 데이터를 바탕으로 응답자 간 유사도 계산 (유사도는 코사인 또는 피어슨 상관계수 사용)
-> 벡터로 변환하여, 이를 통해 유사한 응답자를 찾는 방법

**예시**: 응답자 ID A과 B가 비슷한 스타일을 선호하거나 비선호한다면 두 응답자는 높은 유사도를 갖는다.

응답자 A와 B가 다음과 같은 스타일 선호도를 가진다고 가정
- A: punk, grunge 스타일 선호
- B: minimal, grunge 스타일 선호
응답자 A와 B의 선호도가 유사하다면, B의 선택을 바탕으로 A에게 minimal 스타일을 추천하거나 선호할 가능성이 있다고 예측할 수 있다.

**장점**
- 직관적이고, 유사 사용자 간의 관계를 반영하여 개인화된 추천이 가능
- 선호하는 스타일이 비슷한 사용자 그룹이 있으면 매우 효과적

**단점**
- 데이터가 부족할 경우, 즉, 새로운 사용자나 스타일에 대한 데이터가 없을 경우에는 사용자 선호도나 유사도를 측정하기 어려운 상황이 발생한다. = 유사도를 측정할 데이터가 없기에 추천이 어렵다.
- 많은 사용자에 대한 유사도를 계산해야 하므로 계산 비용이 높다.

***Item-based Filtering (아이템 기반 필터링)***: 응답자가 선호하는 스타일과 유사한 스타일을 추천하는 방식으로, 선호했던 스타일 정보를 바탕으로 유사함을 예측한다.

**적용 방법**: 응답자의 스타일 선호 데이터를 바탕으로 응답자의 각 선호 스타일 간의 유사도를 측정 (유사도는 코사인 또는 피어슨 상관계수 사용) -> 벡터로 변환하여, 이를 통해 유사한 스타일을 찾는 방법

**예시**: 응답자 B가 'minimal' 스타일을 선호할 경우, 이와 유사한 스타일이라고 확인된 'punk' 스타일이나 'grunge'스타일을 추천하는 방식으로 응답자 B가 해당 스타일을 선호할 것이라고 예측할 수 있다.

**장점**
- 새로운 응답자가 있을 경우, 기존 스타일의 유사성을 바탕으로 추천할 수 있기 때문에 데이터 부족 문제를 해결하는 데 유리하다.
- 스타일이 고정적이기 때문에 계산 비용이 낮다/ 안정적이고 다양한 추천을 제공

**단점**
- 응답자 개별 취향을 충분하게 반영 못 할 수 있다.
- 특정 스타일에 대한 선호가 강한 응답자의 특이성을 반영 못할 수 있다.
=> 맞춤 추천이 제한적



**아이템 기반 필터링이 데이터가 부족해도 사용 가능한 이유 **

만약 응답자가 bohemian 스타일을 선호하면, 그와 유사한 hippie나 vintage 스타일을 추천할 수 있습니다. 이 추천 과정은 사용자 간의 관계가 아니라 스타일(아이템) 간의 관계를 기반으로 하기 때문이다.



3-2

학습된 모델 파일과 스타일 선호도 정보표 구글 드라이브에 미리 업로드하여 사용.

In [ ]:
import re  # 경로 수정에 사용
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import os
import ast
from multiprocessing import Pool, cpu_count

In [ ]:
# 학습된 모델의 중간 레이어에서 특징 벡터를 추출하는 클래스 정의
class FeatureExtractor(nn.Module):
    def __init__(self, model):
        super(FeatureExtractor, self).__init__()
        # 마지막 레이어를 제외하고 중간 레이어만 사용하여 특징 벡터 추출
        self.features = nn.Sequential(*list(model.children())[:-1])

    def forward(self, x):
        x = self.features(x)
        # 출력 벡터를 1차원으로 변환하여 반환
        return x.view(x.size(0), -1)

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 사전 학습된 모델 로드
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 2)  # 최종 클래스 개수에 맞춰 설정
model.load_state_dict(torch.load('/content/drive/MyDrive/dataset/resnet18_fashion_classification_with_crops.pth', map_location=device))
model.eval()
model = model.to(device)  # 모델을 GPU로 이동

# 특징 벡터 추출을 위한 모델 정의
feature_extractor = FeatureExtractor(model).to(device)

# 이미지 전처리 설정: 리사이즈, 텐서 변환, 정규화 적용
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5498, 0.5226, 0.5052], std=[0.3301, 0.3257, 0.3320])
])

# CSV 파일 로드 및 경로 보정
preference_df = pd.read_csv("/content/drive/MyDrive/dataset/top_100_respondents_style_preference.csv")
preference_df = preference_df.fillna("")  # 결측값을 빈 문자열로 채움

# CSV 파일의 경로 데이터를 리스트로 파싱하기 위한 함수 정의
def parse_path_column(column):
    # 각 셀의 문자열이 리스트 형식으로 시작하는 경우 이를 파싱하여 리스트로 변환
    return column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("[") else [])

# CSV 파일에서 이미지 경로 데이터를 리스트로 변환
preference_df['train_preferred'] = parse_path_column(preference_df['train_스타일선호'].astype(str))
preference_df['train_non_preferred'] = parse_path_column(preference_df['train_스타일비선호'].astype(str))
preference_df['valid_preferred'] = parse_path_column(preference_df['valid_스타일선호'].astype(str))
preference_df['valid_non_preferred'] = parse_path_column(preference_df['valid_스타일비선호'].astype(str))

# 이미지 경로의 마지막 부분에서 '_숫자' 제거
def clean_path(path):
    return re.sub(r'_\d+(?=\.\w+$)', '', path)

# 이미지를 메모리에 로드하는 함수 정의
def load_image_to_memory(path):
    # 경로 보정 후 이미지 불러오기
    actual_path = "/content/drive/MyDrive/" + clean_path(path)
    try:
        image = Image.open(actual_path).convert('RGB')
        # 불러온 이미지를 변환하고 반환
        return (path, transform(image))
    except FileNotFoundError:
        print(f"File not found: {actual_path}")
        return (path, None)  # 파일이 없을 경우 None 반환

# 메모리에 로드된 이미지를 GPU에서 특징 벡터로 변환
def extract_features_from_memory(feature_extractor, image_dict):
    feature_dict = {}
    for path, image_tensor in image_dict.items():
        if image_tensor is not None:
            # 이미지를 GPU로 이동
            image_tensor = image_tensor.unsqueeze(0).to(device)
            # 특징 벡터 추출
            with torch.no_grad():
                feature_vector = feature_extractor(image_tensor).cpu().numpy()  # 특징 벡터를 CPU로 이동하여 저장
            feature_dict[path] = feature_vector
    return feature_dict

# 전체 이미지를 메모리에 로드하여 특징 벡터를 추출하는 함수
def load_images_and_extract_features(df):
    # 모든 이미지 경로 수집
    image_paths = set()
    for _, row in df.iterrows():
        train_preferred_paths = row['train_preferred']
        train_non_preferred_paths = row['train_non_preferred']
        valid_preferred_paths = row['valid_preferred']
        valid_non_preferred_paths = row['valid_non_preferred']
        # 중복 제거를 위해 set에 추가
        image_paths.update(train_preferred_paths + train_non_preferred_paths + valid_preferred_paths + valid_non_preferred_paths)

    # 병렬로 이미지 로드 및 변환
    with Pool(cpu_count()) as pool:
        images_in_memory = pool.map(load_image_to_memory, image_paths)

    # 로드된 이미지를 딕셔너리 형태로 저장
    image_dict = {path: tensor for path, tensor in images_in_memory if tensor is not None}

    # 특징 벡터 추출
    return extract_features_from_memory(feature_extractor, image_dict)

# 이미지 로드 및 특징 벡터 추출
feature_dict = load_images_and_extract_features(preference_df)

# 유사도를 계산하고 선호도를 예측하는 함수
def predict_preference(preference_df, feature_dict):
    results = []
    similarity_threshold = 0.7  # 유사도 임계값 설정

    for _, row in preference_df.iterrows():
        respondent_id = row['respondent_id']  # 응답자 ID
        # train 이미지의 선호 및 비선호 특징 벡터 수집
        train_preferred_vectors = [feature_dict[path] for path in row['train_preferred'] if path]
        train_non_preferred_vectors = [feature_dict[path] for path in row['train_non_preferred'] if path]

        # validation 이미지들에 대한 예측 수행
        for valid_type in ['valid_preferred', 'valid_non_preferred']:
            valid_paths = row[valid_type]  # 각 validation 이미지 경로
            for valid_path in valid_paths:
                # 유사도 계산을 위해 validation 이미지의 특징 벡터를 가져옴
                valid_vector = feature_dict[valid_path]
                # 각 train 선호 및 비선호 벡터와의 유사도 계산
                preferred_similarities = [cosine_similarity(valid_vector, vec)[0][0] for vec in train_preferred_vectors]
                non_preferred_similarities = [cosine_similarity(valid_vector, vec)[0][0] for vec in train_non_preferred_vectors]

                # 선호도 예측
                max_preferred_sim = max(preferred_similarities) if preferred_similarities else 0
                max_non_preferred_sim = max(non_preferred_similarities) if non_preferred_similarities else 0
                predicted_preference = max_preferred_sim >= max_non_preferred_sim and max_preferred_sim >= similarity_threshold

                # 결과 기록
                results.append({
                    'respondent_id': respondent_id,
                    'valid_image': valid_path,
                    'predicted_preference': predicted_preference,
                    'actual_preference': valid_type == 'valid_preferred'
                })

    # 결과를 데이터프레임으로 저장
    results_df = pd.DataFrame(results)
    results_df.to_csv("predicted_style_preference.csv", index=False)
    print("예측 결과가 저장되었습니다: predicted_style_preference.csv")

# 예측 함수 실행
predict_preference(preference_df, feature_dict)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-17-a44e85e1de2e>:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the function

예측 결과가 저장되었습니다: predicted_style_preference.csv


In [ ]:
results_df = pd.read_csv("/content/predicted_style_preference.csv")
results_df

,respondent_id,valid_image,predicted_preference,actual_preference
0,64747,dataset/validation_image/W_38588_19_genderless...,True,True
1,64747,dataset/validation_image/W_44330_10_sportiveca...,True,True
2,64747,dataset/validation_image/W_37491_70_military_W...,True,True
3,64747,dataset/validation_image/W_22510_80_powersuit_...,True,True
4,64747,dataset/validation_image/W_30988_90_kitsch_W_1...,False,True
...,...,...,...,...
1096,64295,dataset/validation_image/W_32314_19_normcore_M...,True,False
1097,64295,dataset/validation_image/W_25761_90_hiphop_M_1...,False,False
1098,64295,dataset/validation_image/W_31478_19_normcore_M...,False,False
1099,64295,dataset/validation_image/W_16374_10_sportiveca...,False,False


In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# 응답자별 성능 계산
performance_data = []

# 각 응답자에 대해 정밀도, 재현률, F1 점수 계산
for respondent_id, group in results_df.groupby('respondent_id'):
    y_true = group['actual_preference']
    y_pred = group['predicted_preference']

    # 성능 지표 계산 (zero_division=0 추가)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    # 결과 저장
    performance_data.append({
        'respondent_id': respondent_id,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    })

# 성능 데이터프레임 생성
performance_df = pd.DataFrame(performance_data)
performance_df.to_csv("/content/preference_score.csv", index=False)

In [ ]:
# 결과 출력
performance_df

,respondent_id,precision,recall,f1_score
0,368,1.000000,0.833333,0.909091
1,837,1.000000,0.600000,0.750000
2,7658,1.000000,0.333333,0.500000
3,7905,1.000000,0.500000,0.666667
4,9096,0.666667,0.400000,0.500000
...,...,...,...,...
95,66469,0.900000,0.900000,0.900000
96,66513,0.333333,1.000000,0.500000
97,66592,0.800000,1.000000,0.888889
98,66731,0.000000,0.000000,0.000000
